In [1]:
import MySQLdb as db
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Make a list containing lists with all possible combinations of features

In [2]:
# http://stackoverflow.com/questions/464864/python-code-to-pick-out-all-possible-combinations-from-a-list
import itertools

# All possible combinations of features
subsets = []
features = ['Day', 'Time', 'Module', 'Room', 'NumReg', 'Capacity']
for L in range(1, len(features)+1):
    for subset in itertools.combinations(features, L):
        subsets.append(subset)       

# 2. Connect to database - query database for ABT (one for all wifi data and one for averages)

In [70]:
# Connect to Database
name = "DatabaseMain"
con = db.connect(host="localhost", user="root", passwd='', db=name)
cursor = con.cursor()

# ABT - all wifi data into a df
sql = """SELECT G.DateTime, W.Room, R.Capacity, T.Module, M.NumReg, W.Associated, G.PercentageEstimate, G.BinaryEstimate
        FROM DatabaseMain.WifiLogData W, DatabaseMain.GroundTruth G, DatabaseMain.Rooms R, DatabaseMain.TimeModule T, DatabaseMain.Modules M
        WHERE W.Room = G.Room AND W.DateTime BETWEEN G.DateTime AND DATE_ADD(G.DateTime, INTERVAL 1 HOUR) AND R.Room = W.Room AND R.Room = G.Room AND T.Room = G.Room AND T.Room = R.Room AND T.Room =  W.Room AND T.DateTime = G.DateTime AND M.ModuleName = T.Module"""
df = pd.read_sql_query(sql, con)

# ABT - wifi averages into a df
sql_avgs = """SELECT G.DateTime, W.Room, R.Capacity, T.Module, M.NumReg, W.AvgNumWifiConn, G.PercentageEstimate, G.BinaryEstimate
        FROM DatabaseMain.AverageNumWifiConnections W, DatabaseMain.GroundTruth G, DatabaseMain.Rooms R, DatabaseMain.TimeModule T, DatabaseMain.Modules M
        WHERE W.Room = G.Room AND W.DateTime = G.DateTime AND W.Room = G.Room AND W.Room = R.Room AND W.Room = T.Room AND R.Room = W.Room AND R.Room = G.Room AND T.Room = G.Room AND T.Room = R.Room AND T.Room =  W.Room AND T.DateTime = G.DateTime AND M.ModuleName = T.Module
        ORDER BY W.Room"""
df_avgs = pd.read_sql_query(sql_avgs, con)

# 3. Deal with categorical features 
- Split DateTime column and form two new columns (Time and Day) Index by day of week and time of day
- Change type of categorical features to 'category' - this allows us to pick the categorical features out later so can 'get_dummies' (one-hot encoding)
- note: for days of week index starts at 0 (day 0 = Monday, day 1 = Tuesday ...)

In [71]:
# Index DateTime column - want separate day of week and time
# so can be separated by category (e.g day of week or time)

df['Day'] = df['DateTime'].dt.dayofweek
df['Time'] = df['DateTime'].dt.time

df_avgs['Day'] = df_avgs['DateTime'].dt.dayofweek
df_avgs['Time'] = df_avgs['DateTime'].dt.time

categories = ['Day', 'Time', 'Module', 'Room']
for cat in categories:
    df[cat] = df[cat].astype('category')
    df_avgs[cat] = df_avgs[cat].astype('category')
df.dtypes

DateTime              datetime64[ns]
Room                        category
Capacity                       int64
Module                      category
NumReg                         int64
Associated                     int64
PercentageEstimate           float64
BinaryEstimate                 int64
Day                         category
Time                        category
dtype: object

# 4. Deal with Outliers

In [24]:
# KEEP OUTLIERS IN
# df['Estimate'] = df['Capacity'] * df['PercentageEstimate']
# df_avgs['Estimate'] = df_avgs['Capacity'] * df_avgs['PercentageEstimate']

In [72]:
# ALTER OUTLIERS

# http://chrisalbon.com/python/pandas_create_column_using_conditional.html
# Add estimate column
df['Estimate'] = df['Capacity'] * df['PercentageEstimate']
# Handle outliers - replace them with the NumReg
df['Estimate'].loc[df['Estimate'] > 200] = df['NumReg']

# Add estimate column
df_avgs['Estimate'] = df_avgs['Capacity'] * df_avgs['PercentageEstimate']
# Handle outliers - replace them with the NumReg
df_avgs['Estimate'].loc[df_avgs['Estimate'] > 200] = df_avgs['NumReg']

/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# 5. Functions
- Feature Engineering - find best combination of features for model
- PredictOccupancy - make predictions for the best model
- Mode - all wifi data predicts every five minutes - find most common prediction for each hour

In [73]:
# Find best model: can only use tues-fri wk1, mon-fri wk2, 9-5
def FeatureEngineering(df, target, wifiData):
    results = pd.DataFrame(columns=['Features', 'CrossValScore', 'NullAccuracy'])
    intercept = pd.DataFrame({'Intercept':np.ones(len(df))})
    y = df[target]
    
    for ind, sub in enumerate(subsets): 
        x = pd.concat([intercept, df[wifiData]], axis=1)
        features = [wifiData]
        for s in sub:
            features.append(s)
            if pd.core.common.is_categorical_dtype(df[s]):
                x = pd.concat([x, pd.get_dummies(df[s], prefix=s)], axis=1)
            else:
                x = pd.concat([x, df[s]], axis=1)
        results.loc[ind] = [features, cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv = 10).mean(), cross_val_score(DummyClassifier(strategy='most_frequent'), x, y, scoring='accuracy', cv = 10).mean()]
    return results

def PredictOccupancy(results, df, target, wifiData):
    features = results['Features'][results['CrossValScore'].idxmax()]
    x = pd.DataFrame(pd.concat([pd.DataFrame({'Intercept':np.ones(len(df))})], axis=1))
    y = df[target]

    for feat in features:
        if pd.core.common.is_categorical_dtype(df[feat]):
            x = pd.concat([x, pd.get_dummies(df[feat], prefix=feat)], axis=1)
        else:
            x = pd.concat([x, df[feat]], axis=1)
  
    logreg = LogisticRegression().fit(x, y)
    predictions = pd.DataFrame(logreg.predict(x), columns=["Predictions"])
    return pd.concat([df['DateTime'], df['Room'], predictions], axis=1)

def Mode(df, predictions):
    ''' all wifi data predicts every five minutes but we want only one prediction for each hour 
        take most common result of each hour as final prediction'''
    # list of all datetimes
    DateTimes = df['DateTime'].unique()
    # list of all rooms
    Rooms = df['Room'].unique()

    # dataframes to hold results
    final_predictions = pd.DataFrame(columns=['DateTime', 'Room', 'Estimate'])
    room = pd.DataFrame(columns=['DateTime', 'Room', 'Estimate'])

    # iterate through each datetime for eech room
    for r in Rooms:
        for ind, dt in enumerate(DateTimes):
            # get all the predictions for particular hour for particular room
            # http://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
            group = predictions['Predictions'].loc[((predictions['DateTime'] == dt) & (predictions['Room'] == r))]
        #     print(group['Predictions'].value_counts())
        #     print('MAX:', group['Predictions'].value_counts().idxmax())
            # add datetime, room and most common prediction for the hour to a dataframe
            # most frequent value: http://stackoverflow.com/questions/15138973/how-to-get-the-number-of-the-most-frequent-value-in-a-column
            room.loc[ind] = [dt, r, group.value_counts().idxmax()]
        # add dataframe (room) containing final predictions for a certain room to the final_predictions df
        # need to do this becuase 'ind' gets reset for each room so will overwrite rows already in dataframe 'room'
        final_predictions = final_predictions.append(room, ignore_index=True)
    return final_predictions

# 6.  BINARY - FIND BEST MODEL, PREDICT VALUES AND SAVE TO TABLE

In [34]:
target = 'BinaryEstimate'
wifiData = 'Associated'
wifiDataAvgs = 'AvgNumWifiConn'

# FeatureEngineering returns a df with all possible models,
# df contains colums with features used in the model, the cross validation score and the null_accuracy cross validation score
res = FeatureEngineering(df, target, wifiData)
res_avgs = FeatureEngineering(df_avgs, target, wifiDataAvgs)

In [8]:
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.799403984064
Averages:  0.852077922078
Features in best model (all wifi data):  ['Associated', 'Time', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Room', 'Capacity']


In [35]:
# OUTLIERS REMOVED
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.799403984064
Averages:  0.852077922078
Features in best model (all wifi data):  ['Associated', 'Time', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Room', 'Capacity']


In [36]:
# # Make predictions for best model for all wifi data and best model for average wifi data
# predictions = PredictOccupancy(res, df, target, wifiData)
# predictions_avgs = PredictOccupancy(res_avgs, df_avgs, target, wifiDataAvgs)

# select if all wifi data or wifi averages has the best model, make predictions for that model only
# pick best model between all wifi data and wifi averages
df = pd.DataFrame(df if res['CrossValScore'].max() > res_avgs['CrossValScore'].max() else df_avgs)
res = pd.DataFrame(res if 'Associated' in df.columns else res_avgs)
wifiData = wifiData if 'Associated' in df.columns else wifiDataAvgs
print(wifiData)

    
predictions = PredictOccupancy(res, df, target, wifiData)
if wifiData == 'Associated':
    predictions = Mode(df, predictions)
    
predictions.to_sql(con=con, name='BinaryPredictions', if_exists='replace', flavor='mysql')

AvgNumWifiConn


/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/pandas/core/generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


# 7. PERCENTAGE - FIND BEST MODEL, PREDICT VALUES AND SAVE TO TABLE

In [65]:
# RUN SECTION 2-4 AGAIN FIRST TO GET ORIGINAL ABTS
# Predicting for percentage - get target column

df['EstimateAsPercent'] = df['Estimate'] / df['Capacity']
groups = [ '0%', '25%', '50%', '75%', '100%',]
bins = [-0.01, 0.00, 0.25, 0.50, 0.75, 1.00]
df['PercentagePred'] = pd.cut(df['EstimateAsPercent'], bins, labels = groups )
df['PercentageCat'] = df[['PercentagePred', 'Room']].apply(lambda x: ''.join(x), axis=1)
df = df.drop(['PercentagePred', 'EstimateAsPercent'], axis = 1)


df_avgs['EstimateAsPercent'] = df_avgs['Estimate'] / df_avgs['Capacity']
groups = [ '0%', '25%', '50%', '75%', '100%',]
bins = [-0.01, 0.00, 0.25, 0.50, 0.75, 1.00]
df_avgs['PercentagePred'] = pd.cut(df_avgs['EstimateAsPercent'], bins, labels = groups )
df_avgs['PercentageCat'] = df_avgs[['PercentagePred', 'Room']].apply(lambda x: ''.join(x), axis=1)
df_avgs = df_avgs.drop(['PercentagePred', 'EstimateAsPercent'], axis = 1)

In [66]:
target = 'PercentageCat'
wifiData = 'Associated'
wifiDataAvgs = 'AvgNumWifiConn'

# FeatureEngineering returns a df with all possible models,
# df contains colums with features used in the model, the cross validation score and the null_accuracy cross validation score
res = FeatureEngineering(df, target, wifiData)
res_avgs = FeatureEngineering(df_avgs, target, wifiDataAvgs)

/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: War

In [41]:
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.596718935469
Averages:  0.62951756037
Features in best model (all wifi data):  ['Associated', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Day', 'Room', 'NumReg']


In [68]:
# OUTLIERS REMOVED
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.597589863514
Averages:  0.63210750746
Features in best model (all wifi data):  ['Associated', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Day', 'Room', 'NumReg']


In [69]:
# # Make predictions for best model for all wifi data and best model for average wifi data
# predictions = PredictOccupancy(res, df, target, wifiData)
# predictions_avgs = PredictOccupancy(res_avgs, df_avgs, target, wifiDataAvgs)

# select if all wifi data or wifi averages has the best model, make predictions for that model only
# pick best model between all wifi data and wifi averages
df = pd.DataFrame(df if res['CrossValScore'].max() > res_avgs['CrossValScore'].max() else df_avgs)
res = pd.DataFrame(res if 'Associated' in df.columns else res_avgs)
wifiData = wifiData if 'Associated' in df.columns else wifiDataAvgs
print(wifiData)

    
predictions = PredictOccupancy(res, df, target, wifiData)
if wifiData == 'Associated':
    predictions = Mode(df, predictions)

predictions['Predictions'] = pd.DataFrame(predictions['Predictions'][i].split('%')[0] for i in range(0, len(predictions)))
predictions.to_sql(con=con, name='PercentagePredictions', if_exists='replace', flavor='mysql')

AvgNumWifiConn


/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/pandas/core/generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


# 8. ESTIMATE - FIND BEST MODEL, PREDICT VALUES AND SAVE TO TABLE

In [74]:
# RUN SECTION 2-4 AGAIN FIRST TO GET ORIGINAL ABTS
# Bin results into categories for logistic regression. 
bins = [-1, 25, 50, 75, 100, 125, 150, 175, 200, 220]
groups = [ '0-25', '25-50', '50-75', '75-100', '100-125', '125-150', '150-175', '175-200', '200-220']
df['OccupantEstimate'] = pd.cut(df['Estimate'], bins, labels = groups )
df_avgs['OccupantEstimate'] = pd.cut(df_avgs['Estimate'], bins, labels = groups )

In [75]:
target = 'OccupantEstimate'
wifiData = 'Associated'
wifiDataAvgs = 'AvgNumWifiConn'

# FeatureEngineering returns a df with all possible models,
# df contains colums with features used in the model, the cross validation score and the null_accuracy cross validation score
res = FeatureEngineering(df, target, wifiData)
res_avgs = FeatureEngineering(df_avgs, target, wifiDataAvgs)

/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/sklearn/cross_validation.py:516: War

In [73]:
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.697665560283
Averages:  0.719772256729
Features in best model (all wifi data):  ['Associated', 'Time', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Day', 'Module', 'NumReg', 'Capacity']


In [76]:
# OUTLIERS REMOVED
# Find best model (highest cross val score)
print("All wifi data: ", res['CrossValScore'].max())
print("Averages: ", res_avgs['CrossValScore'].max())
print("Features in best model (all wifi data): ", res['Features'][res['CrossValScore'].idxmax()] )
print("Features in best model (averages): ", res_avgs['Features'][res_avgs['CrossValScore'].idxmax()] )

All wifi data:  0.697665560283
Averages:  0.719772256729
Features in best model (all wifi data):  ['Associated', 'Time', 'Room', 'Capacity']
Features in best model (averages):  ['AvgNumWifiConn', 'Day', 'Module', 'NumReg', 'Capacity']


In [78]:
# # Make predictions for best model for all wifi data and best model for average wifi data
# predictions = PredictOccupancy(res, df, target, wifiData)
# predictions_avgs = PredictOccupancy(res_avgs, df_avgs, target, wifiDataAvgs)

# select if all wifi data or wifi averages has the best model, make predictions for that model only
# pick best model between all wifi data and wifi averages
df = pd.DataFrame(df if res['CrossValScore'].max() > res_avgs['CrossValScore'].max() else df_avgs)
res = pd.DataFrame(res if 'Associated' in df.columns else res_avgs)
wifiData = wifiData if 'Associated' in df.columns else wifiDataAvgs
print(wifiData)

    
predictions = PredictOccupancy(res, df, target, wifiData)
if wifiData == 'Associated':
    predictions = Mode(df, predictions)

predictions.to_sql(con=con, name='EstimatePredictions', if_exists='replace', flavor='mysql')

AvgNumWifiConn


/Users/JoanMcCarthy/anaconda/envs/project/lib/python3.5/site-packages/pandas/core/generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)
